In [26]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [75]:
library(Metrics)
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

df.training <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource, -SalePrice)
    
df.testing  <- 
    df.combined %>% 
    filter(dataSource == "test") %>% 
    select(-dataSource, -SalePrice)

In [45]:
partition <- caret::createDataPartition(y=df.training$sale_price_log, p=.5, list=F, times=1)

df.training.train <- df.training[partition,]
df.training.test <- df.training[-partition,]

tran.res <- kaggle.house$trans$doItAll(df.training.train, df.training.test)
df.training.train.final <- tran.res$df.training
df.training.test.final <- tran.res$df.testing

# Should be only numeric columns
stopifnot(0 == df.training.train.final %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)
stopifnot(0 == df.training.test.final %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)

c(df.training.train %>% nrow, df.training.test %>% nrow)

[1] 731 729

In [51]:
model <- lm(sale_price_log ~ ., data=df.training.train.final)

glance(model)
tidy(model) %>% arrange(p.value) %>% filter(p.value < 0.1)

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.931911,0.9239985,0.1112546,117.7773,0,77,608.6659,-1061.332,-702.9676,8.094944,654


term,estimate,std.error,statistic,p.value
X2ndFlrSF,2.717688e-04,2.548269e-05,10.664837,1.361547e-24
OverallQual,5.422069e-02,6.550716e-03,8.277064,7.103108e-16
X1stFlrSF,2.741403e-04,3.345567e-05,8.194136,1.330530e-15
MSZoning.new,2.725639e-01,3.397643e-02,8.022147,4.813287e-15
OverallCond,4.441166e-02,5.746008e-03,7.729134,4.093416e-14
SaleCondition.new,3.261553e-01,6.731244e-02,4.845394,1.579511e-06
is_full_functional,8.075182e-02,1.816063e-02,4.446532,1.025234e-05
Condition1.new,2.350959e-01,5.516020e-02,4.262057,2.323601e-05
BsmtFinSF1,1.166406e-04,3.035094e-05,3.843064,1.333591e-04
PoolArea,2.184194e-03,6.669953e-04,3.274677,1.113625e-03


In [76]:
y_test_predicted <- predict(model, df.training.test.final) %>% as.vector
y_test_actual <- df.training.test.final[, "sale_price_log"] %>% `[[`(1)

mean(100 * abs(y_test_actual - y_test_predicted) / y_test_actual)

rmse(y_test_actual, y_test_predicted)

Warning message in predict.lm(model, df.training.test.final):
“prediction from a rank-deficient fit may be misleading”

[1] 0.7405244

[1] 0.1565868

In [12]:
partitions <- caret::createDataPartition(y=df.training$sale_price_log, p=.5, list=F, times=20)

trains <- partitions %>% as.data.frame %>% 
          purrr::map(function (sample) { lm(sale_price_log ~ ., data=df.training[sample,]) })
tests <- partitions %>% as.data.frame %>% 
          purrr::map(function (sample) { df.training[-sample,] })

In [9]:
trains %>% purrr::map(function (model) { summary(model)$r.squared }) %>% unlist %>% mean

[1] 0.9253752

In [22]:
mod <- trains[[1]]
te <- tests[[1]]

y_test_predicted <- predict(mod, te)
y_test_actual <- te[, "sale_price_log"] %>% `[[`(1)
y_test_predicted %>% as.vector

y_test_actual

Warning message in predict.lm(mod, te):
“prediction from a rank-deficient fit may be misleading”

[1] 12.22181 12.16788 12.52233 12.44474 11.62212 11.59742 12.32593 11.53089
  [9] 11.80491 12.35958 11.82679 11.90510 11.80138 12.14633 11.24519 11.85257
 [17] 12.07322 12.54080 12.62881 11.94795 11.77065 11.16771 12.01462 11.93364
 [25] 12.55917 12.39742 11.28894 12.73201 11.67702 12.96411 11.75254 11.81902
 [33] 12.60450 11.46606 11.43897 11.66143 11.99666 12.07789 12.06254 11.63821
 [41] 11.88365 11.65667 11.99997 12.41829 11.48847 11.60198 12.37280 12.32934
 [49] 12.09579 11.96861 11.43181 12.08598 11.44455 11.79491 11.88771 11.94386
 [57] 12.28470 12.27691 12.25879 11.57624 12.39294 11.86230 11.53614 12.68423
 [65] 12.12735 12.08303 11.64754 12.60068 11.95188 11.71190 12.23324 12.26570
 [73] 11.61177 11.52866 11.93798 12.85625 12.12961 12.47465 12.17005 11.71980
 [81] 11.93892 12.68701 12.22008 11.72750 12.65416 11.69785 11.52146 11.97816
 [89] 12.42972 11.93190 11.78399 12.24260 11.31576 12.25434 12.09054 11.58297
 [97] 12.89102 11.62677 12.79884 11.41173 12.34445 12.50549 12.43997 11.61447
[105] 12.35947 12.33328 11.89958 12.15348 11.37953 12.06749 11.85994 12.30949
[113] 12.20525 12.26795 11.45955 11.98172 11.99811 11.65421 12.12132 12.12745
[121] 12.50335 12.13114 11.72093 12.20550 12.25054 12.28972 12.18621 12.26241
[129] 12.16098 11.60532 11.94655 12.43134 11.77264 12.35623 11.96700 12.48358
[137] 12.29099 12.69236 11.55834 11.55451 11.82883 12.95906 12.12885 12.47374
[145] 12.45596 12.79612 12.44700 11.65399 12.54975 11.39275 12.43117 11.88303
[153] 11.42530 11.85912 12.22145 12.27853 11.97299 12.27792 11.75635 12.14210
[161] 12.17241 12.21535 11.54392 11.61295 12.03692 11.82055 11.97721 11.69976
[169] 11.98744 12.07437 12.03283 12.07366 11.54218 11.76024 11.72394 12.58858
[177] 12.67340 12.12563 11.87648 12.21084 11.38160 12.42928 12.18155 11.81422
[185] 12.18662 12.98376 12.30989 11.62636 11.74907 11.22348 12.27094 12.37384
[193] 12.34824 11.71961 12.01803 12.67297 11.56135 12.39850 11.56084 12.36392
[201] 12.16102 11.91345 12.35204 11.39908 11.83244 12.31583 11.89794 11.85251
[209] 11.61059 12.19076 12.18200 11.53174 12.06553 12.33729 11.66617 11.63090
[217] 11.89662 12.23659 12.31071 11.98568 11.56267 12.20989 12.02599 11.44689
[225] 12.20196 11.69192 12.41623 12.05659 11.70759 12.05508 12.28633 11.77042
[233] 12.78449 11.83063 11.59056 12.73944 12.75551 11.97111 11.88390 11.87991
[241] 11.64095 11.89580 11.49833 11.06861 12.00234 12.23896 11.89211 12.44351
[249] 11.64015 12.12578 11.93172 11.95107 11.64022 11.90784 11.45639 12.88895
[257] 12.45138 11.95629 11.89491 13.93143 11.79547 11.48670 12.41383 12.12706
[265] 10.86752 11.81581 11.66257 11.79728 12.68762 12.19190 11.76354 12.24893
[273] 12.05518 11.54458 12.34457 11.88557 11.58200 12.47494 12.20235 11.98374
[281] 11.97326 11.67686 11.81178 12.20699 12.51705 11.80019 11.93185 11.68358
[289] 12.30963 11.96522 11.63639 12.73927 11.73065 12.79101 11.69157 11.72994
[297] 12.45853 12.13799 11.81868 11.69935 12.69680 11.67998 12.32398 12.25183
[305] 11.84260 11.78213 11.99405 12.29935 12.27002 11.93179 12.18804 12.62009
[313] 11.69129 12.75811 11.88740 12.66239 12.45305 11.94419 12.05450 11.89307
[321] 11.80721 11.77449 11.52864 12.49204 12.38077 12.82976 11.85927 11.80933
[329] 12.01782 11.28424 11.61528 12.16835 11.42603 11.85026 12.90452 12.27372
[337] 11.92708 11.56575 12.03358 12.49770 11.69325 11.59703 12.13501 12.25306
[345] 12.33595 11.99798 12.45573 11.86357 13.51265 12.59005 11.55721 11.46327
[353] 11.87194 11.84110 12.17815 11.92059 12.70459 12.30500 11.28693 11.59284
[361] 12.21849 11.81912 12.49901 11.83014 12.58276 11.66379 12.40494 12.16373
[369] 11.49303 12.41557 12.17822 11.58556 11.88233 11.65901 12.07469 12.32249
[377] 11.32484 12.00842 12.48374 11.94329 11.98509 12.24651 12.03805 12.57616
[385] 11.72945 12.07355 12.68514 12.40748 12.42105 12.04308 11.75072 11.73056
[393] 12.59685 11.98001 12.34622 11.76100 12.02880 11.72952 12.31479 11.49088
[401] 12.42023 12.18909 12.01366 11.70793 12.16294 13.05148 11.56207 11.73605
[409] 11.80002 11.75405 

[1] 12.24769 12.10901 12.63460 12.20607 11.77452 11.67844 12.54076 11.40756
  [9] 11.84510 12.34583 11.77452 11.94471 11.81155 12.24289 10.59663 11.91405
 [17] 12.01673 12.53358 12.64110 11.93819 11.59910 11.31447 12.04355 11.87757
 [25] 12.67576 12.42802 11.60824 12.86100 11.77529 12.99175 11.73527 11.84940
 [33] 12.66666 11.58432 11.41861 11.60824 12.03469 12.06681 12.01067 11.49883
 [41] 12.00457 11.45900 12.04061 12.42922 11.51293 11.65269 12.28765 12.46651
 [49] 12.00756 12.10071 11.51293 12.10625 11.34451 11.75979 11.95118 11.91839
 [57] 12.32829 12.40492 12.30138 11.65269 12.46844 11.73607 11.65269 12.82773
 [65] 12.15478 12.36734 11.73607 12.67608 11.93164 11.75587 12.15478 12.33710
 [73] 11.76368 11.51293 12.20657 13.07107 12.06105 12.66033 12.12269 11.75194
 [81] 11.90834 12.65035 12.36734 11.55215 12.52416 11.84940 11.49272 12.13350
 [89] 12.43987 11.95761 11.80895 12.25486 11.58058 12.23028 12.10016 11.48247
 [97] 12.86424 11.73607 12.90669 11.40199 12.17819 12.66981 12.47801 11.61277
[105] 12.23077 12.39462 11.84940 12.10071 11.24505 12.06105 11.88449 12.34583
[113] 12.24289 12.35234 11.48247 12.07824 11.92504 11.77529 12.07539 12.12811
[121] 12.49125 12.39462 11.73206 12.21106 12.16525 12.33929 12.12811 12.24289
[129] 12.09626 11.71178 11.94194 12.35978 11.78295 12.38422 11.97351 12.49500
[137] 12.23077 12.59473 11.40199 11.32055 11.79056 12.83468 12.14685 12.46844
[145] 12.50618 12.74444 12.61487 11.74543 12.39669 11.37366 12.68850 11.88621
[153] 11.26446 11.68688 12.24047 12.27373 11.95118 12.22047 11.85083 11.96718
[161] 11.94471 12.15478 11.46163 11.57025 12.06105 11.80560 11.95761 11.88449
[169] 12.19854 11.97666 11.99535 12.05757 11.80881 11.73607 11.71994 12.73670
[177] 12.88520 12.09514 11.75194 12.27139 11.23849 12.38839 12.16525 11.73607
[185] 12.16003 12.96219 12.27839 11.59910 11.71994 11.11245 12.39255 12.41105
[193] 12.46071 11.65269 12.08391 12.73597 11.00210 12.31043 11.65269 12.33710
[201] 12.10699 11.91505 12.38422 11.74404 11.86358 12.27839 11.84223 11.81303
[209] 11.59918 12.18280 12.07254 11.28853 12.10625 12.26434 11.66135 11.68688
[217] 11.60824 12.05815 12.25486 11.75587 11.69525 12.22588 12.14420 11.49272
[225] 12.45293 11.60824 12.48156 11.95118 11.72804 12.02575 12.15478 11.90497
[233] 12.99453 11.79434 11.40167 12.69465 12.83201 12.00762 11.89819 11.95761
[241] 11.65269 11.79810 11.42191 10.46024 12.12269 12.33138 11.84940 12.57418
[249] 11.73206 12.24673 11.98916 12.01309 11.77452 11.80560 11.47730 12.90635
[257] 12.48749 11.91839 11.97666 12.12676 11.79056 11.36210 12.20919 12.07254
[265] 10.57898 11.58525 11.61954 11.97035 12.66033 12.27022 11.79810 12.09885
[273] 12.25486 11.73607 12.47991 11.84940 11.63071 12.55673 12.07254 11.70767
[281] 12.04355 11.58989 11.75979 12.27373 12.66539 11.82012 11.86780 11.69525
[289] 12.32163 11.89136 11.78676 12.44230 11.79810 12.82099 11.77529 11.82774
[297] 11.87060 12.13296 11.84940 11.60824 12.67295 11.64837 12.17665 12.28995
[305] 11.92504 11.85652 11.92504 12.30592 12.23077 11.93164 12.32386 12.79136
[313] 11.68267 12.65396 11.83138 12.62807 12.38839 11.98293 11.93819 12.03469
[321] 11.84510 11.87757 11.35041 12.52088 12.32829 12.75130 11.93164 11.87235
[329] 11.95118 11.34451 11.58989 12.16003 11.38509 11.83138 12.95513 12.17303
[337] 12.03172 11.54829 12.01370 12.45877 11.60368 11.97939 12.06105 12.24289
[345] 12.33655 11.91036 12.87902 11.85652 13.53447 12.72189 11.59432 11.39639
[353] 11.72400 11.83863 12.18587 11.84940 12.61818 12.44509 11.54031 11.76757
[361] 11.97980 11.76368 12.52453 11.81303 12.67608 11.69941 12.31043 12.17819
[369] 11.54248 12.37370 12.14153 11.58989 12.00151 11.79056 12.09514 12.37159
[377] 11.49272 11.99535 12.52634 11.95761 12.05815 12.26434 12.09737 12.57764
[385] 11.75587 12.27932 12.72784 12.50618 12.48420 11.98293 11.81229 11.53273
[393] 12.88664 11.99535 12.30818 11.86710 12.07824 11.75979 12.35879 11.58896
[401] 12.50540 12.17819 12.07254 11.60733 12.14950 13.27583 11.67844 11.81673
[409] 11.98262 11.57119 

In [99]:
?union

In [98]:
trains %>% purrr::map(function (model) { tidy(model) %>% filter(p.value < 0.2) %>% select(term) }) 

term
LotArea
OverallQual
OverallCond
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
X1stFlrSF
X2ndFlrSF
BsmtFullBath
FullBath


In [88]:
x <- tidy(trains[[1]])
x %>% filter(p.value < 0.2)

term,estimate,std.error,statistic,p.value
LotArea,1.137148e-06,8.137010e-07,1.397501,1.627377e-01
OverallQual,5.936847e-02,7.585392e-03,7.826685,2.025456e-14
OverallCond,3.650917e-02,6.190921e-03,5.897212,5.932018e-09
X1stFlrSF,2.753192e-04,3.505241e-05,7.854499,1.654241e-14
X2ndFlrSF,2.258774e-04,3.048919e-05,7.408444,3.965819e-13
LowQualFinSF,1.889051e-04,1.240109e-04,1.523293,1.281694e-01
BsmtFullBath,5.675407e-02,1.465853e-02,3.871745,1.189279e-04
HalfBath,2.105185e-02,1.471698e-02,1.430447,1.530672e-01
GarageCars,5.788703e-02,1.631102e-02,3.548952,4.145159e-04
WoodDeckSF,7.293203e-05,4.520412e-05,1.613394,1.071422e-01


In [36]:
partition <- createDataPartition(y=df.training$sale_price_log, p=.5, list=F)

df.training.train <- df.training[partition,]
df.training.test <- df.training[-partition,]

model_1 = lm(sale_price_log ~ ., data=df.training.train)
summary(model_1)


Call:
lm(formula = sale_price_log ~ ., data = df.training.train)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.75363 -0.05871  0.00750  0.06838  0.44689 

Coefficients: (3 not defined because of singularities)
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)            -2.953e+00  8.267e+00  -0.357 0.721072    
LotFrontage             6.121e-05  1.561e-04   0.392 0.695181    
LotArea                 2.327e-06  8.798e-07   2.646 0.008352 ** 
OverallQual             5.658e-02  7.322e-03   7.727 4.15e-14 ***
OverallCond             3.682e-02  6.270e-03   5.873 6.83e-09 ***
YearBuilt              -6.675e-05  4.484e-04  -0.149 0.881707    
YearRemodAdd            7.230e-04  3.893e-04   1.857 0.063764 .  
MasVnrArea             -5.209e-06  3.565e-05  -0.146 0.883863    
BsmtFinSF1              4.606e-05  2.787e-05   1.652 0.098924 .  
BsmtFinSF2              4.480e-05  4.082e-05   1.098 0.272716    
BsmtUnfSF               1.815e-05  2.696e-05   0

In [37]:
df.training2.train <- df.training2[partition,]
df.training2.test <- df.training2[-partition,]

model_2 = lm(sale_price_log ~ ., data=df.training2.train)
summary(model_2)


Call:
lm(formula = sale_price_log ~ ., data = df.training2.train)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.74502 -0.05587  0.00842  0.06595  0.46684 

Coefficients: (1 not defined because of singularities)
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)            -2.610e+00  8.193e+00  -0.319 0.750204    
LotFrontage            -3.977e-05  1.574e-04  -0.253 0.800583    
LotArea                 5.362e-02  1.365e-02   3.929 9.44e-05 ***
OverallQual             5.952e-02  7.287e-03   8.168 1.63e-15 ***
OverallCond             3.603e-02  6.174e-03   5.835 8.47e-09 ***
YearBuilt              -4.086e-05  4.453e-04  -0.092 0.926909    
YearRemodAdd            7.995e-04  3.851e-04   2.076 0.038300 *  
MasVnrArea             -6.683e-07  3.546e-05  -0.019 0.984969    
BsmtFinSF1             -9.541e-06  6.949e-05  -0.137 0.890831    
BsmtFinSF2             -1.197e-03  6.016e-03  -0.199 0.842323    
BsmtUnfSF              -4.282e-05  6.992e-05  -